In [1]:
import telebot
from telebot import types
import os
import logging
import time
import datetime
import threading
import sqlite3
import re
from cryptography.fernet import Fernet
import hashlib
import uuid
from dotenv import load_dotenv
load_dotenv()

# Set up logging
logging.basicConfig(level=logging.INFO)

# Local directory to store files
local_storage_dir = '/scratch/project_2004147/telebot'
voice_files_dir = os.path.join(local_storage_dir, 'voice_messages')

# Ensure directories exist
os.makedirs(local_storage_dir, exist_ok=True)
os.makedirs(voice_files_dir, exist_ok=True)

# Telegram bot token
TOKEN = os.environ.get('TELEGRAM_BOT_TOKEN')
bot = telebot.TeleBot(TOKEN, threaded=True)

# Retrieve the bot's username
bot_info = bot.get_me()
bot_username = bot_info.username

# User data storage
user_data = {}
user_profiles = {}

# Initialize locks for thread safety
db_lock = threading.Lock()


# Database file path
db_file = os.path.join(local_storage_dir, 'responses.db')


# Load environment variables from .env file
load_dotenv()

# Retrieve the encryption key from the environment variable
ENCRYPTION_KEY_GLOBAL = os.environ.get('ENCRYPTION_KEY')

if ENCRYPTION_KEY_GLOBAL:
    # Encode the key to bytes
    ENCRYPTION_KEY_GLOBAL = ENCRYPTION_KEY_GLOBAL.encode()
    # Validate the key
    try:
        fernet = Fernet(ENCRYPTION_KEY_GLOBAL)
    except ValueError as e:
        raise ValueError(f"Invalid encryption key: {e}")
else:
    raise ValueError("No encryption key provided. Set the ENCRYPTION_KEY environment variable.")


In [2]:
# Placeholders for web links
PRIVACY_NOTICE_URL_EN = 'https://telegra.ph/Ukrainabilitys-Privacy-Notice-10-13'
PARTICIPANT_INFORMATION_SHEET_URL_EN = 'https://telegra.ph/Participant-Information-Sheet-10-14'

PRIVACY_NOTICE_URL_UK = 'https://telegra.ph/Pol%D1%96tika-konf%D1%96denc%D1%96jnost%D1%96-dlya-dosl%D1%96dzhennya-Ukrainability-10-13'
PARTICIPANT_INFORMATION_SHEET_URL_UK = 'https://telegra.ph/%D0%86nformac%D1%96jnij-list-dlya-uchasnik%D1%96v-10-14'

# Message templates for English and Ukrainian
messages = {
    'en': {
        'welcome': "Welcome! Please select a language.",
        'select_language': "Please select a language:",
        'language_selected': "Language set to English.",
        'project_intro': f'''Participation confirmation
This survey is conducted by Aalto University (Finland) and the Urban Reform agency (Ukraine) to understand people's outdoor experiences in Ukraine. By sharing your experiences, you help us study nature-related values and the impact of landscape changes over time. Please do not participate if you are currently residing in 1) temporarily occupied territories or 2) areas near the frontline. Participants should not send information or media files related to military activities. The survey is for persons aged 18 or older.

Participation is voluntary and at any point in the research study, you are at liberty to stop if you no longer wish to participate in the study, but all the information gathered up until that point can be used as described in the <a href="{PRIVACY_NOTICE_URL_EN}">privacy notice</a> and <a href="{PARTICIPANT_INFORMATION_SHEET_URL_EN}">participant information sheet</a>.

By clicking 'I agree', I confirm that I am 18 years or older, have received sufficient information about the research study, have the opportunity to answer the questions, have understood the information, and wish to participate in the study.

Do you agree to participate and share your data for research purposes? Your data will be processed and stored in compliance with GDPR and Ukrainian personal data protection law.''',
        'consent_options': ["I agree", "I do not agree"],
        'consent_given': "Thank you for agreeing to participate!",
        'consent_denied': "Thank you for your time. If you change your mind, you can restart the bot.",
        'restart_button': "Restart",
        'send_location': (
            "Please send the location of a place where you spent some time outdoors;"
            " if the place is too large, select an approximate location.\n\n"
            "To do this:\n"
            "1. Tap the attachment icon (📎) in the message bar.\n"
            "2. Select 'Location'.\n"
            "3. Move the map to the desired location.\n"
            "4. Tap 'Send this location'."
        ),
        'enjoyment_question': "How was your time spent at this place?",
        'enjoyment_options': [
            "Very enjoyable",
            "Enjoyable",
            "Neutral",
            "Not enjoyable",
            "Not enjoyable at all"
        ],
        'invalid_rating': "Invalid input. Please select an option requested in the question.",
        'purpose_visit': "What was the purpose(s) of your visit to this location? You can select multiple options.",
        'other_purpose': "Please specify your purpose of visit:",
        'regularity_question': "How often did you visit this place over the last years?",
        'frequency_change_question': "Is this frequency different compared to previous years?",
        'changes_question': "Have you noticed any changes in this place after the full-scale invasion?",
        'changes_detail_question': "What exactly has changed? You can select multiple options.",
        'other_changes_detail': "Please specify what else has changed:",
        'add_description': (
            "We'd love to hear more about your experience. Please share any stories, feelings, or memorable moments from your visit. "
            "You can type your response or send a voice message. The voice message will be transcribed to text for analysis, and the original message will be deleted immediately afterward to protect your privacy. If you'd prefer to skip and finish, please press 'Skip'."
        ),
        'thank_you': "Thank you! Your data has been received.",
        'thank_you_lottery': "Thank you! Your data has been received. Good luck in the lottery!",
        'thank_you_no_lottery': "Thank you! Your data has been received.",
        'skip_button': "Skip",
        'age_question': "Please select your age group:",
        'gender_question': "Please select your gender:",
        'occupation_question': "Please select your occupational status:",
        'income_question': "Please select your monthly income level:",
        'done_button': "Done",
        'lottery_question': (
            "Would you like a chance to win a voucher for one of Ukrainian online stores worth 1000 UAH? "
            "If yes, we will ask for your contact e-mail."
        ),
        'lottery_options': ["Yes", "No"],
        'email_request': "Please enter your contact e-mail:",
        'invalid_email': "Invalid email format. Please enter a valid email address:",
        'travel_time_question': "How long did it take you to travel to this location from your place of residence?",
        'continue_question': "Would you like to submit another place or stop here?",
        'continue_options': ["Continue", "Stop"],
        'location_received': "Location received",
        'responses_so_far': "Your responses so far:",
        'description_skipped': "Description skipped.",
        'your_description': "Your description:",
        'selected': "Selected:",
        'unselected': "Unselected:",
        'you_selected': "You selected:",
        'voice_message_submitted': "Voice message submitted.",
        'email_recorded': "Your email has been recorded as: {email}",
        'error_occurred': "An error occurred. Please try again later.",
        'invalid_selection': "Invalid selection.",
        'please_select_at_least_one': "Please select at least one option.",
        'options': {
            'purpose_visit': [
                "Walking",
                "Jogging/Yoga, etc.",
                "Cycling",
                "Pet walking",
                "Socialising",
                "Bird watching",
                "Fishing",
                "Photography",
                "Mushroom/Berry picking",
                "Picnicking",
                "Relaxing",
                "Other (what?)"
            ],
            'regularity': [
                "Very frequently",
                "Frequently",
                "Occasionally",
                "Rarely",
                "This was my first visit",
                "I didn’t visit this place last year"
            ],
            'frequency_change': [
                "Yes, I visit more often now",
                "Yes, I visit less often now or stopped visiting",
                "No, it's about the same"
            ],
            'noticed_changes': [
                "Yes, positive changes",
                "Yes, negative changes",
                "No noticeable changes"
            ],
            'changes_detail': [
                "Landscape beauty",
                "Quality of recreation",
                "Accessibility",
                "Intactness/stewardship",
                "Public facilities",
                "Wildlife and vegetation",
                "Safety",
                "Other (what?)"
            ],
            'age': [
                "18-25", "26-40", "41-60", "Above 60", "Prefer not to disclose"
            ],
            'gender': [
                "Male", "Female", "Other", "Prefer not to disclose"
            ],
            'occupation': [
                "Working", "Not working", "Student", "Retired", "Military service", "Other", "Prefer not to disclose"
            ],
            'income': [
                "0-5000 UAH", "5001-10000 UAH", "10001-20000 UAH", "More than 20000 UAH", "Prefer not to disclose"
            ],
            'travel_time': [ 
                "Less than 30 minutes",
                "30 minutes to 1 hour",
                "1-2 hours",
                "More than 2 hours",
                "Prefer not to say"
            ],
        },
    },
    'uk': {
        'welcome': "Ласкаво просимо! Будь ласка, оберіть мову.",
        'select_language': "Будь ласка, оберіть мову:",
        'language_selected': "Мову встановлено на українську.",
        'project_intro': f'''Підтвердження участі
Це опитування Університету Аалто (Фінляндія) та агенції Urban Reform (Україна), має на меті зрозуміти роль довкілля у повсякденному житті та активному відпочинку українців. Ділячись своїм досвідом, ви допомагаєте нам вивчати цінності, пов’язані із перебуванням на природі, і відповідний вплив змін ландшафту. Будь ласка, не беріть участь в опитуванні, якщо ви зараз знаходитесь на 1) тимчасово окупованих або 2) прифронтових територіях України. Крім того, учасникам суворо не рекомендується надсилати будь-яку інформацію чи медіа-файли, безпосередньо пов’язані з воєнною активністю з будь-якої сторони. Це опитування не призначене для осіб молодших 18 років.

Участь є добровільною, і в будь-який момент дослідження ви маєте право відмовитись від участі, якщо більше не бажаєте брати участь у дослідженні, але вся інформація, зібрана до цього моменту, може бути використана, як описано в <a href="{PRIVACY_NOTICE_URL_UK}">політиці конфіденційності</a> та <a href="{PARTICIPANT_INFORMATION_SHEET_URL_UK}">інформаційному листі учасника</a>.

Натискаючи «Я погоджуюсь», я підтверджую, що отримав(ла) достатньо інформації про дослідження, я маю можливість надати відповіді на запитання, я розумію інформацію наведену вище та бажаю взяти участь у дослідженні.

Чи згодні ви брати участь і ділитися своїми даними для дослідницьких цілей? Ваші дані будуть оброблятись та зберігатись відповідно до законодавства ЄС про дані General Data Protection Regulation (GDPR) та українського Закону про захист персональних даних. ''',
        'consent_options': ["Я згоден/згодна", "Я не згоден/не згодна"],
        'consent_given': "Дякуємо за вашу згоду на участь!",
        'consent_denied': "Дякуємо за ваш час. Якщо ви передумаєте, ви можете перезапустити бота.",
        'restart_button': "Перезапустити",
        'send_location': (
            "Будь ласка, надішліть місце, де провели якийсь час на свіжому повітрі;"
            " якщо це місце велике за площею, просто оберіть приблизну локацію.\n\n"
            "Щоб зробити це:\n"
            "1. Натисніть на іконку вкладення (📎) у рядку повідомлень.\n"
            "2. Оберіть 'Розташування'.\n"
            "3. Перемістіть карту до бажаного місця.\n"
            "4. Натисніть 'Надіслати вибране розташування'."
        ),
        'enjoyment_question': "Як вам сподобався ваш час, проведений у цьому місці?",
        'enjoyment_options': [
            "Дуже сподобався",
            "Сподобався",
            "Нейтрально",
            "Не сподобався",
            "Зовсім не сподобався"
        ],
        'invalid_rating': "Недійсний вибір. Будь ласка, оберіть варіант, запропонований у запитанні.",
        'purpose_visit': "Якою була мета вашого візиту до цього місця? Ви можете обрати декілька варіантів.",
        'other_purpose': "Будь ласка, вкажіть вашу мету візиту:",
        'regularity_question': "Впродовж останніх 12 місяців, як часто ви відвідували це місце?",
        'frequency_change_question': "Чи відрізняється ця частота від попередніх років?",
        'changes_question': "Чи помітили ви якісь зміни в цьому місці після повномасштабного вторгнення?",
        'changes_detail_question': "Що саме змінилося? Ви можете обрати декілька варіантів.",
        'other_changes_detail': "Будь ласка, вкажіть, що ще змінилося:",
        'add_description': (
            "Ми були б раді почути більше про ваш досвід. Будь ласка, поділіться будь-якими історіями, почуттями або пам'ятними моментами від вашого візиту. "
            "Ви можете написати відповідь або надіслати голосове повідомлення. Голосове повідомлення буде перетворено в текст для аналізу, а оригінальне голосове повідомлення буде негайно видалене для захисту вашої конфіденційності. Якщо ви хочете пропустити та завершити, будь ласка, натисніть 'Пропустити'."
        ),
        'thank_you': "Дякуємо! Ваші дані були отримані.",
        'thank_you_lottery': "Дякуємо! Ваші дані були отримані. Бажаємо успіху в розіграші!",
        'thank_you_no_lottery': "Дякуємо! Ваші дані були отримані.",
        'skip_button': "Пропустити",
        'age_question': "Будь ласка, оберіть вашу вікову групу:",
        'gender_question': "Будь ласка, оберіть вашу стать:",
        'occupation_question': "Будь ласка, оберіть ваш основний професійний статус:",
        'income_question': "Будь ласка, оберіть рівень вашого місячного доходу:",
        'done_button': "Готово",
        'lottery_question': (
            "Чи бажаєте ви отримати шанс виграти ваучер на суму 1000 грн для одного з українських онлайн-магазинів? "
            "Якщо так, ми попросимо вас залишити вашу контактну електронну адресу."
        ),
        'lottery_options': ["Так", "Ні"],
        'email_request': "Будь ласка, введіть вашу контактну електронну адресу:",
        'invalid_email': "Недійсний формат електронної адреси. Будь ласка, введіть дійсну електронну адресу:",
        'travel_time_question': "Скільки часу вам знадобилося, щоб дістатися до цього місця зі свого місця проживання?",
        'continue_question': "Ви бажаєте розповісти про інше місце чи завершити?",
        'continue_options': ["Продовжити", "Завершити"],
        'location_received': "Локацію отримано",
        'responses_so_far': "Ваші відповіді наразі:",
        'description_skipped': "Опис пропущено.",
        'your_description': "Ваш опис:",
        'selected': "Вибрано:",
        'unselected': "Знято вибір:",
        'you_selected': "Ви обрали:",
        'voice_message_submitted': "Голосове повідомлення надіслано.",
        'email_recorded': "Ваша електронна адреса: {email} була збережена.",
        'error_occurred': "Виникла помилка. Будь ласка, спробуйте пізніше.",
        'invalid_selection': "Недійсний вибір.",
        'please_select_at_least_one': "Будь ласка, оберіть принаймні один варіант.",
        'options': {
            'purpose_visit': [
                "Прогулянка",
                "Пробіжка, йога, тощо",
                "Катання на велосипеді",
                "Вигул домашніх тварин",
                "Спілкування",
                "Спостереження за птахами",
                "Риболовля",
                "Фотографування",
                "Збирання грибів/ягід",
                "Пікнік",
                "Відпочинок",
                "Інше (що саме?)"
            ],
            'regularity': [
                "Дуже часто",
                "Часто",
                "Час від часу",
                "Рідко",
                "Вперше",
                "Не відвідував(ла)"
            ],
            'frequency_change': [
                "Так, зараз я відвідую частіше",
                "Так, зараз я відвідую рідше чи зовсім не відвідую",
                "Ні, приблизно та сама"
            ],
            'noticed_changes': [
                "Так, позитивні зміни",
                "Так, негативні зміни",
                "Не помітив(ла) жодних змін"
            ],
            'changes_detail': [
                "Краса пейзажу",
                "Якість відпочинку",
                "Зручність доступу",
                "Збереженість/догляд",
                "Громадські об'єкти",
                "Природа та рослинність",
                "Відчуття безпеки",
                "Інше (що саме?)"
            ],
            'age': [
                "18-25", "26-40", "41-60", "Понад 60", "Віддаю перевагу не вказувати"
            ],
            'gender': [
                "Чоловіча", "Жіноча", "Інше", "Віддаю перевагу не вказувати"
            ],
            'occupation': [
                "Працюю", "Не працюю", "Навчаюсь", "На пенсії", "Військова служба", "Інше", "Віддаю перевагу не вказувати"
            ],
            'income': [
                "0-5000 грн", "5001-10000 грн", "10001-20000 грн", "Більше 20000 грн", "Віддаю перевагу не вказувати"
            ],
            'travel_time': [ 
                "Менше 30 хвилин",
                "30 хвилин - 1 година",
                "1-2 години",
                "Більше 2 годин",
                "Віддаю перевагу не вказувати"
            ],
        },
    }
}


In [3]:
def initialize_database():
    with db_lock:
        try:
            conn = sqlite3.connect(db_file, check_same_thread=False)
            cursor = conn.cursor()

            # Create table if not exists
            create_table_query = '''
                CREATE TABLE IF NOT EXISTS responses (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_id TEXT,
                    latitude TEXT,
                    longitude TEXT,
                    enjoyment TEXT,
                    purpose_visit TEXT,
                    regularity TEXT,
                    noticed_changes TEXT,
                    changes_detail TEXT,
                    other_changes_detail TEXT,
                    travel_time TEXT,
                    description TEXT,
                    voice_submitted TEXT,
                    age TEXT,
                    gender TEXT,
                    occupation TEXT,
                    income TEXT,
                    language TEXT,
                    email TEXT,
                    timestamp TEXT
                );
            '''
            cursor.execute(create_table_query)
            conn.commit()

            # Check existing columns
            cursor.execute("PRAGMA table_info(responses)")
            existing_columns = [info[1] for info in cursor.fetchall()]

            # List of required columns
            required_columns = [
                'user_id', 'latitude', 'longitude', 'enjoyment', 'purpose_visit',
                'regularity', 'frequency_change', 'noticed_changes', 'changes_detail',
                'other_changes_detail', 'travel_time', 'description', 'voice_submitted',
                'age', 'gender', 'occupation', 'income', 'language', 'email', 'timestamp'
            ]

            # Add missing columns
            for column in required_columns:
                if column not in existing_columns:
                    cursor.execute(f"ALTER TABLE responses ADD COLUMN {column} TEXT;")
                    conn.commit()

        except Exception as e:
            logging.exception(f"Error initializing database: {e}")
            raise e
        finally:
            conn.close()


In [4]:
def create_inline_keyboard(options, prefix):
    """
    Creates an InlineKeyboardMarkup with buttons based on the provided options.

    Args:
        options (list): List of option strings.
        prefix (str): Prefix for callback_data to identify the question.

    Returns:
        InlineKeyboardMarkup: The generated inline keyboard.
    """
    try:
        inline_kb = types.InlineKeyboardMarkup(row_width=2)
        buttons = [
            types.InlineKeyboardButton(text=option, callback_data=f"{prefix}_{idx}")
            for idx, option in enumerate(options)
        ]
        inline_kb.add(*buttons)
        return inline_kb
    except Exception as e:
        logging.exception(f"Error in create_inline_keyboard: {e}")
        return types.InlineKeyboardMarkup()  # Return an empty keyboard to prevent further errors


In [5]:
def escape_html(text):
    """
    Escapes HTML special characters in text.
    """
    if not isinstance(text, str):
        text = str(text)
    return text.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')


In [6]:
@bot.message_handler(commands=['start'])
def send_welcome(message=None, chat_id=None, user_id=None):
    try:
        if message:
            chat_id = message.chat.id
            user_id = message.from_user.id
            # Check if /start has parameters
            if message.text.startswith('/start '):
                start_param = message.text.split(' ', 1)[1]
            else:
                start_param = None
        elif chat_id and user_id:
            start_param = None
        else:
            # Cannot proceed without chat_id and user_id
            return

        # Initialize user_data[user_id] if not present
        if user_id not in user_data:
            user_data[user_id] = {}

        if start_param == 'restart':
            # Reset experience-related data in user_data[user_id], but keep language and profile data
            # Remove keys related to the previous experience
            keys_to_remove = ['location', 'enjoyment', 'purpose_visit', 'regularity', 'frequency_change', 'noticed_changes',
                              'changes_detail', 'other_changes_detail', 'travel_time', 'description', 'voice_submitted']
            for key in keys_to_remove:
                user_data[user_id].pop(key, None)
        else:
            # Fresh start or /start without parameters
            # Do not reset user_data[user_id], just proceed
            pass

        # Check if user has already provided language
        if user_id in user_profiles and 'language' in user_profiles[user_id]:
            language = user_profiles[user_id]['language']
            user_data[user_id]['language'] = language
            # Proceed directly to sending location
            bot.send_message(chat_id, messages[language]['send_location'])
        else:
            # Send welcome message and ask for language selection using InlineKeyboard
            inline_kb = types.InlineKeyboardMarkup(row_width=2)
            english_button = types.InlineKeyboardButton('English', callback_data='language_en')
            ukrainian_button = types.InlineKeyboardButton('Українська', callback_data='language_uk')
            inline_kb.add(english_button, ukrainian_button)

            bot.send_message(
                chat_id,
                messages['en']['welcome'] + "\n" + messages['uk']['welcome'],
                reply_markup=inline_kb
            )
    except Exception as e:
        logging.exception(f"Error in send_welcome: {e}")
        bot.send_message(chat_id, "An error occurred. Please try again later.")


In [7]:
@bot.callback_query_handler(func=lambda call: call.data.startswith('language_'))
def handle_language_selection(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id
        data = call.data.split('_')[1]
        
        if data == 'en':
            user_data[user_id]['language'] = 'en'
        elif data == 'uk':
            user_data[user_id]['language'] = 'uk'
        else:
            # Invalid selection
            bot.answer_callback_query(call.id, "Invalid selection.")
            return

        language = user_data[user_id]['language']
        # Acknowledge the selection
        bot.answer_callback_query(call.id, f"Language set to {language.upper()}.")

        # Remove the inline keyboard
        bot.edit_message_reply_markup(chat_id=chat_id, message_id=call.message.message_id, reply_markup=None)

        # Confirm language selection
        bot.send_message(chat_id, messages[language]['language_selected'])

        # Store language in user_profiles
        user_profiles.setdefault(user_id, {})['language'] = language

        # Check if user has already given consent
        if user_id in user_profiles and 'consent' in user_profiles[user_id]:
            # Proceed to send location request
            bot.send_message(chat_id, messages[language]['send_location'])
        else:
            # Send introductory message and request consent using InlineKeyboard
            options = messages[language]['consent_options']
            inline_kb = types.InlineKeyboardMarkup(row_width=2)
            buttons = [
                types.InlineKeyboardButton(text=option, callback_data=f"consent_{idx}")
                for idx, option in enumerate(options)
            ]
            inline_kb.add(*buttons)
            bot.send_message(
                chat_id,
                messages[language]['project_intro'],
                parse_mode='HTML',
                reply_markup=inline_kb
            )
    except Exception as e:
        logging.exception(f"Error in handle_language_selection: {e}")
        bot.send_message(chat_id, "An error occurred. Please try again later.")


In [8]:
@bot.callback_query_handler(func=lambda call: call.data.startswith('consent_'))
def handle_consent(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id
        idx = int(call.data.split('_')[1])
        language = user_data[user_id]['language']
        consent_options = messages[language]['consent_options']

        if 0 <= idx < len(consent_options):
            consent_response = consent_options[idx]
            # Remove the inline keyboard
            bot.edit_message_reply_markup(chat_id=chat_id, message_id=call.message.message_id, reply_markup=None)
            if consent_response == consent_options[0]:  # User agrees
                bot.send_message(chat_id, messages[language]['consent_given'])
                # Store consent in user_profiles
                user_profiles.setdefault(user_id, {})['consent'] = True
                # Proceed to send location request
                bot.send_message(chat_id, messages[language]['send_location'])
            elif consent_response == consent_options[1]:  # User does not agree
                # Construct the restart URL using the bot's username with a 'restart' parameter
                restart_url = f"https://t.me/{bot_username}?start=restart"

                # Create an inline keyboard with the "Restart" button
                inline_kb = types.InlineKeyboardMarkup()
                restart_button = types.InlineKeyboardButton(
                    text=messages[language]['restart_button'],
                    url=restart_url
                )
                inline_kb.add(restart_button)

                # Send the 'consent_denied' message with the restart button
                bot.send_message(
                    chat_id,
                    messages[language]['consent_denied'],
                    reply_markup=inline_kb
                )

                # Clear user data
                if user_id in user_data:
                    del user_data[user_id]
            else:
                # Invalid input
                bot.answer_callback_query(call.id, messages[language]['invalid_selection'])
        else:
            # Invalid index
            bot.answer_callback_query(call.id, messages[language]['invalid_selection'])
    except Exception as e:
        logging.exception(f"Error in handle_consent: {e}")
        bot.send_message(chat_id, "An error occurred. Please try again later.")


In [9]:
@bot.message_handler(content_types=['location'])
def handle_location(message):
    try:
        chat_id = message.chat.id
        user_id = message.from_user.id

        # Check if the user has selected a language
        if user_id not in user_data or 'language' not in user_data[user_id]:
            # Use stored language from user_profiles
            if user_id in user_profiles and 'language' in user_profiles[user_id]:
                user_data[user_id] = {'language': user_profiles[user_id]['language']}
            else:
                # Ask the user to start again
                bot.send_message(chat_id, "Please use /start to begin.\nБудь ласка, використайте /start для початку.")
                return

        language = user_data[user_id]['language']

        latitude = message.location.latitude
        longitude = message.location.longitude

        # Store location data
        user_data[user_id]['location'] = {'latitude': latitude, 'longitude': longitude}

        # Acknowledge the received location
        bot.send_message(chat_id, messages[language]['location_received'])

        # Proceed to enjoyment question
        ask_enjoyment(chat_id, user_id, language)
    except Exception as e:
        logging.exception(f"Error in handle_location: {e}")
        bot.reply_to(message, "An error occurred. Please try again later.")


In [10]:
def ask_purpose_visit(chat_id, user_id, language):
    try:
        user_data[user_id]['purpose_visit'] = []
        options = messages[language]['options']['purpose_visit']
        inline_kb = types.InlineKeyboardMarkup(row_width=2)
        
        buttons = [
            types.InlineKeyboardButton(text=option, callback_data=f"purpose_{idx}")
            for idx, option in enumerate(options)
        ]
        done_button = types.InlineKeyboardButton(text=messages[language]['done_button'], callback_data="purpose_done")
        inline_kb.add(*buttons)
        inline_kb.add(done_button)
        
        bot.send_message(
            chat_id,
            messages[language]['purpose_visit'],
            reply_markup=inline_kb
        )
    except Exception as e:
        logging.exception(f"Error in ask_purpose_visit: {e}")
        bot.send_message(chat_id, messages[language].get('error_occurred', "An error occurred. Please try again later."))


In [11]:
@bot.callback_query_handler(func=lambda call: call.data.startswith('purpose_'))
def handle_purpose_selection(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id
        language = user_data[user_id]['language']
        data = call.data.split('_')[1]

        if data == 'done':
            if not user_data[user_id]['purpose_visit']:
                bot.answer_callback_query(call.id, messages[language].get('please_select_at_least_one', "Please select at least one option."))
                return

            # Remove the inline keyboard
            bot.edit_message_reply_markup(chat_id=chat_id, message_id=call.message.message_id, reply_markup=None)

            # Echo the user's selections in italics
            purposes = '; '.join(escape_html(purpose) for purpose in user_data[user_id]['purpose_visit'])
            bot.send_message(chat_id, f"<i>{purposes}</i>", parse_mode='HTML')

            # Get the 'Other' option text from the messages dictionary
            other_option = messages[language]['options']['purpose_visit'][-1]
            # Check if 'Other (what?)' was selected to prompt for additional input
            if other_option in user_data[user_id]['purpose_visit']:
                msg = bot.send_message(
                    chat_id,
                    messages[language]['other_purpose']
                )
                bot.register_next_step_handler(msg, handle_other_purpose)
            else:
                # Proceed to regularity question
                ask_regularity(chat_id, user_id, language)
        else:
            idx = int(data)
            options = messages[language]['options']['purpose_visit']
            if 0 <= idx < len(options):
                selected_option = options[idx]
                if selected_option in user_data[user_id]['purpose_visit']:
                    user_data[user_id]['purpose_visit'].remove(selected_option)
                    bot.answer_callback_query(call.id, f"{messages[language]['unselected']} {selected_option}")
                else:
                    user_data[user_id]['purpose_visit'].append(selected_option)
                    bot.answer_callback_query(call.id, f"{messages[language]['selected']} {selected_option}")

                # Update the inline keyboard to reflect current selections
                update_purpose_selection_keyboard(call.message, user_id, language)
            else:
                bot.answer_callback_query(call.id, messages[language].get('invalid_selection', "Invalid selection."))
    except Exception as e:
        logging.exception(f"Error in handle_purpose_selection: {e}")
        bot.send_message(chat_id, messages[language].get('error_occurred', "An error occurred. Please try again later."))


In [12]:
def handle_other_purpose(message):
    try:
        chat_id = message.chat.id
        user_id = message.from_user.id
        language = user_data[user_id]['language']
        other_purpose = message.text.strip()

        # Store the user's specified purpose
        user_data[user_id]['purpose_visit'].append(other_purpose)

        # Echo the user's input in italics
        bot.send_message(chat_id, f"<i>{escape_html(other_purpose)}</i>", parse_mode='HTML')

        # Proceed to regularity question
        ask_regularity(chat_id, user_id, language)
    except Exception as e:
        logging.exception(f"Error in handle_other_purpose: {e}")
        bot.reply_to(message, "An error occurred. Please try again later.")


In [13]:
def ask_enjoyment(chat_id, user_id, language, remove_keyboard=False):
    try:
        options = messages[language]['enjoyment_options']
        inline_kb = types.InlineKeyboardMarkup(row_width=1)
        buttons = []
        for idx, option in enumerate(options):
            button = types.InlineKeyboardButton(text=option, callback_data=f"enjoyment_{idx}")
            buttons.append(button)
        inline_kb.add(*buttons)
        bot.send_message(
            chat_id,
            messages[language]['enjoyment_question'],
            reply_markup=inline_kb
        )
    except Exception as e:
        logging.exception(f"Error in ask_enjoyment: {e}")
        bot.send_message(chat_id, "An error occurred. Please try again later.")

In [14]:
@bot.callback_query_handler(func=lambda call: call.data.startswith('enjoyment_'))
def handle_enjoyment_selection(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id

        # Ensure the user has selected a language
        if user_id not in user_data or 'language' not in user_data[user_id]:
            # Use stored language from user_profiles
            if user_id in user_profiles and 'language' in user_profiles[user_id]:
                user_data[user_id] = {'language': user_profiles[user_id]['language']}
            else:
                # Ask the user to start again
                bot.send_message(chat_id, "Please use /start to begin.\nБудь ласка, використайте /start для початку.")
                return

        language = user_data[user_id]['language']
        options = messages[language]['enjoyment_options']
        idx = int(call.data[len('enjoyment_'):])

        if 0 <= idx < len(options):
            enjoyment = options[idx]
            user_data[user_id]['enjoyment'] = enjoyment

            # Acknowledge the selection
            bot.answer_callback_query(call.id, f"{messages[language]['selected']} {enjoyment}")

            # Remove the inline keyboard
            bot.edit_message_reply_markup(chat_id=chat_id, message_id=call.message.message_id, reply_markup=None)

            # Echo the user's selection in italics
            bot.send_message(chat_id, f"<i>{escape_html(enjoyment)}</i>", parse_mode='HTML')

            # Proceed to purpose of visit
            ask_purpose_visit(chat_id, user_id, language)
        else:
            bot.answer_callback_query(call.id, messages[language]['invalid_rating'])
    except Exception as e:
        logging.exception(f"Error in handle_enjoyment_selection: {e}")
        bot.send_message(chat_id, "An error occurred. Please try again later.")


In [15]:
def ask_regularity(chat_id, user_id, language):
    try:
        user_data[user_id]['regularity'] = ''
        options = messages[language]['options']['regularity']
        inline_kb = types.InlineKeyboardMarkup(row_width=2)
        
        buttons = [
            types.InlineKeyboardButton(text=option, callback_data=f"regularity_{idx}")
            for idx, option in enumerate(options)
        ]
        inline_kb.add(*buttons)
        
        bot.send_message(
            chat_id,
            messages[language]['regularity_question'],
            reply_markup=inline_kb
        )
    except Exception as e:
        logging.exception(f"Error in ask_regularity: {e}")
        bot.send_message(chat_id, messages[language].get('error_occurred', "An error occurred. Please try again later."))


In [16]:
@bot.callback_query_handler(func=lambda call: call.data.startswith('regularity_'))
def handle_regularity_selection(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id
        language = user_data[user_id]['language']
        data = call.data.split('_')[1]
        options = messages[language]['options']['regularity']
        idx = int(data)

        if 0 <= idx < len(options):
            selected_regularity = options[idx]
            user_data[user_id]['regularity'] = selected_regularity
            user_profiles.setdefault(user_id, {})['regularity'] = selected_regularity

            # Acknowledge the selection
            bot.answer_callback_query(call.id, f"{messages[language]['selected']} {selected_regularity}")

            # Remove the inline keyboard
            bot.edit_message_reply_markup(chat_id=chat_id, message_id=call.message.message_id, reply_markup=None)

            # Echo the user's selection in italics
            bot.send_message(chat_id, f"<i>{escape_html(selected_regularity)}</i>", parse_mode='HTML')

            # Proceed to ask about frequency change
            ask_frequency_change(chat_id, user_id, language)
        else:
            bot.answer_callback_query(call.id, messages[language].get('invalid_selection', "Invalid selection."))
    except Exception as e:
        logging.exception(f"Error in handle_regularity_selection: {e}")
        bot.send_message(chat_id, messages[language].get('error_occurred', "An error occurred. Please try again later."))


In [17]:
def ask_frequency_change(chat_id, user_id, language):
    try:
        user_data[user_id]['frequency_change'] = ''
        options = messages[language]['options']['frequency_change']
        inline_kb = types.InlineKeyboardMarkup(row_width=1)
    
        buttons = [
            types.InlineKeyboardButton(text=option, callback_data=f"frequency_change_{idx}")
            for idx, option in enumerate(options)
        ]
        inline_kb.add(*buttons)
    
        bot.send_message(
            chat_id,
            messages[language]['frequency_change_question'],
            reply_markup=inline_kb
        )
    except Exception as e:
        logging.exception(f"Error in ask_frequency_change: {e}")
        bot.send_message(
            chat_id,
            messages[language].get('error_occurred', "An error occurred. Please try again later.")
        )


In [18]:
@bot.callback_query_handler(func=lambda call: call.data.startswith('frequency_change_'))
def handle_frequency_change_selection(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id
        language = user_data[user_id]['language']
        data = call.data.split('_')[2]  # Extract the index

        options = messages[language]['options']['frequency_change']

        if data.isdigit():
            idx = int(data)
            if 0 <= idx < len(options):
                selected_frequency_change = options[idx]
                user_data[user_id]['frequency_change'] = selected_frequency_change

                # Acknowledge the selection
                bot.answer_callback_query(call.id, f"{messages[language]['selected']} {selected_frequency_change}")

                # Remove the inline keyboard
                bot.edit_message_reply_markup(chat_id=chat_id, message_id=call.message.message_id, reply_markup=None)

                # Echo the user's selection in italics
                bot.send_message(chat_id, f"<i>{escape_html(selected_frequency_change)}</i>", parse_mode='HTML')

                # Proceed to ask about travel time
                ask_travel_time(chat_id, user_id, language)
            else:
                bot.answer_callback_query(
                    call.id,
                    messages[language].get('invalid_selection', "Invalid selection.")
                )
        else:
            bot.answer_callback_query(
                call.id,
                messages[language].get('invalid_selection', "Invalid selection.")
            )
    except Exception as e:
        logging.exception(f"Error in handle_frequency_change_selection: {e}")
        bot.send_message(
            chat_id,
            messages[language].get('error_occurred', "An error occurred. Please try again later.")
        )


In [19]:
def ask_travel_time(chat_id, user_id, language):
    try:
        user_data[user_id]['travel_time'] = ''
        options = messages[language]['options']['travel_time']
        inline_kb = types.InlineKeyboardMarkup(row_width=1)
        buttons = [
            types.InlineKeyboardButton(text=option, callback_data=f"travel_time_{idx}")
            for idx, option in enumerate(options)
        ]
        inline_kb.add(*buttons)
    
        bot.send_message(
            chat_id,
            messages[language]['travel_time_question'],
            reply_markup=inline_kb
        )
    except Exception as e:
        logging.exception(f"Error in ask_travel_time: {e}")
        bot.send_message(
            chat_id,
            messages[language].get('error_occurred', "An error occurred. Please try again later.")
        )


In [20]:
@bot.callback_query_handler(func=lambda call: call.data.startswith('travel_time_'))
def handle_travel_time_selection(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id
        language = user_data[user_id]['language']
        data = call.data.split('_')[2]  # Correctly split to get the index

        # Validate that 'data' is an integer
        if data.isdigit():
            idx = int(data)
            options = messages[language]['options']['travel_time']

            if 0 <= idx < len(options):
                selected_travel_time = options[idx]
                user_data[user_id]['travel_time'] = selected_travel_time
                user_profiles.setdefault(user_id, {})['travel_time'] = selected_travel_time

                # Acknowledge the selection
                bot.answer_callback_query(call.id, f"{messages[language]['selected']} {selected_travel_time}")

                # Remove the inline keyboard
                bot.edit_message_reply_markup(
                    chat_id=chat_id,
                    message_id=call.message.message_id,
                    reply_markup=None
                )

                # Echo the user's selection in italics
                bot.send_message(chat_id, f"<i>{escape_html(selected_travel_time)}</i>", parse_mode='HTML')

                # Determine the next step based on regularity
                regularity = user_data[user_id].get('regularity', '')
                trigger_regularities = {
                    'en': messages['en']['options']['regularity'][:4],  # First four options
                    'uk': messages['uk']['options']['regularity'][:4]
                }

                if regularity in trigger_regularities.get(language, []):
                    # Ask about noticed changes
                    ask_noticed_changes(chat_id, user_id, language)
                else:
                    # Proceed to socioeconomic questions
                    ask_age(chat_id, user_id, language)
            else:
                bot.answer_callback_query(call.id, messages[language].get('invalid_selection', "Invalid selection."))
        else:
            bot.answer_callback_query(call.id, messages[language].get('invalid_selection', "Invalid selection."))
    except Exception as e:
        logging.exception(f"Error in handle_travel_time_selection: {e}")
        bot.send_message(
            chat_id,
            messages[language].get('error_occurred', "An error occurred. Please try again later.")
        )


In [21]:
def ask_noticed_changes(chat_id, user_id, language):
    try:
        user_data[user_id]['noticed_changes'] = ''
        options = messages[language]['options']['noticed_changes']
        inline_kb = types.InlineKeyboardMarkup(row_width=2)
        buttons = [
            types.InlineKeyboardButton(text=option, callback_data=f"noticed_changes_{idx}")
            for idx, option in enumerate(options)
        ]
        inline_kb.add(*buttons)
        
        bot.send_message(
            chat_id,
            messages[language]['changes_question'],
            reply_markup=inline_kb
        )
    except Exception as e:
        logging.exception(f"Error in ask_noticed_changes: {e}")
        bot.send_message(chat_id, messages[language].get('error_occurred', "An error occurred. Please try again later."))


In [22]:
@bot.callback_query_handler(func=lambda call: call.data.startswith('noticed_changes_'))
def handle_noticed_changes_selection(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id
        language = user_data[user_id]['language']
        data = call.data.split('_')[2]
        options = messages[language]['options']['noticed_changes']
        idx = int(data)

        if 0 <= idx < len(options):
            selected_change = options[idx]
            user_data[user_id]['noticed_changes'] = selected_change
            user_profiles.setdefault(user_id, {})['noticed_changes'] = selected_change

            # Acknowledge the selection
            bot.answer_callback_query(call.id, f"{messages[language]['selected']} {selected_change}")

            # Remove the inline keyboard
            bot.edit_message_reply_markup(chat_id=chat_id, message_id=call.message.message_id, reply_markup=None)

            # Echo the user's selection in italics
            bot.send_message(chat_id, f"<i>{escape_html(selected_change)}</i>", parse_mode='HTML')

            if selected_change in ["Yes, positive changes", "Yes, negative changes", "Так, позитивні зміни", "Так, негативні зміни"]:
                # Proceed to ask for changes detail
                ask_changes_detail(chat_id, user_id, language)
            else:
                # Proceed to socioeconomic questions
                ask_age(chat_id, user_id, language)
        else:
            bot.answer_callback_query(call.id, messages[language].get('invalid_selection', "Invalid selection."))
    except Exception as e:
        logging.exception(f"Error in handle_noticed_changes_selection: {e}")
        bot.send_message(chat_id, messages[language].get('error_occurred', "An error occurred. Please try again later."))


In [23]:
def ask_changes_detail(chat_id, user_id, language):
    try:
        user_data[user_id]['changes_detail'] = []
        options = messages[language]['options']['changes_detail']
        inline_kb = types.InlineKeyboardMarkup(row_width=2)
        
        buttons = [
            types.InlineKeyboardButton(text=option, callback_data=f"changes_detail_{idx}")
            for idx, option in enumerate(options)
        ]
        done_button = types.InlineKeyboardButton(text=messages[language]['done_button'], callback_data="changes_detail_done")
        inline_kb.add(*buttons)
        inline_kb.add(done_button)
        
        bot.send_message(
            chat_id,
            messages[language]['changes_detail_question'],
            reply_markup=inline_kb
        )
    except Exception as e:
        logging.exception(f"Error in ask_changes_detail: {e}")
        bot.send_message(chat_id, messages[language].get('error_occurred', "An error occurred. Please try again later."))


In [24]:
@bot.callback_query_handler(func=lambda call: call.data.startswith('changes_detail_'))
def handle_changes_detail_selection(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id
        language = user_data[user_id]['language']
        data = call.data.split('_')[2]  # Extract the third part after splitting by '_'
        options = messages[language]['options']['changes_detail']

        if data == 'done':
            # User has finished selecting changes details
            if not user_data[user_id]['changes_detail']:
                bot.answer_callback_query(
                    call.id,
                    messages[language].get('please_select_at_least_one', "Please select at least one option.")
                )
                return

            # Remove the inline keyboard from the message
            bot.edit_message_reply_markup(
                chat_id=chat_id,
                message_id=call.message.message_id,
                reply_markup=None
            )

            # Echo the user's selections in italics
            changes = '; '.join(escape_html(change) for change in user_data[user_id]['changes_detail'])
            bot.send_message(chat_id, f"<i>{changes}</i>", parse_mode='HTML')

            # Check if 'Other (what?)' was selected to prompt for additional input
            other_option = messages[language]['options']['changes_detail'][-1]
            if other_option in user_data[user_id]['changes_detail']:
                msg = bot.send_message(
                    chat_id,
                    messages[language]['other_changes_detail']
                )
                bot.register_next_step_handler(msg, handle_other_changes_detail)
            else:
                # Proceed to socioeconomic questions
                ask_age(chat_id, user_id, language)
        else:
            # Attempt to convert `data` to integer only if it's not 'done'
            try:
                idx = int(data)
                if 0 <= idx < len(options):
                    selected_option = options[idx]
                    if selected_option in user_data[user_id]['changes_detail']:
                        # User is unselecting an option
                        user_data[user_id]['changes_detail'].remove(selected_option)
                        bot.answer_callback_query(call.id, f"{messages[language]['unselected']} {selected_option}")
                    else:
                        # User is selecting an option
                        user_data[user_id]['changes_detail'].append(selected_option)
                        bot.answer_callback_query(call.id, f"{messages[language]['selected']} {selected_option}")

                    # Update the inline keyboard to reflect current selections
                    update_changes_detail_selection_keyboard(call.message, user_id, language)
                else:
                    bot.answer_callback_query(
                        call.id,
                        messages[language].get('invalid_selection', "Invalid selection.")
                    )
            except ValueError:
                # `data` was not an integer and not 'done'
                bot.answer_callback_query(
                    call.id,
                    messages[language].get('invalid_selection', "Invalid selection.")
                )
    except Exception as e:
        logging.exception(f"Error in handle_changes_detail_selection: {e}")
        bot.send_message(
            chat_id,
            messages[language].get('error_occurred', "An error occurred. Please try again later.")
        )


In [25]:
def handle_other_changes_detail(message):
    try:
        chat_id = message.chat.id
        user_id = message.from_user.id
        language = user_data[user_id]['language']
        other_changes_detail = message.text.strip()

        # Store the user's specified other changes detail
        user_data[user_id]['other_changes_detail'] = other_changes_detail

        # Echo the user's input in italics
        bot.send_message(chat_id, f"<i>{escape_html(other_changes_detail)}</i>", parse_mode='HTML')

        # Proceed to socioeconomic questions
        ask_age(chat_id, user_id, language)
    except Exception as e:
        logging.exception(f"Error in handle_other_changes_detail: {e}")
        bot.reply_to(message, "An error occurred. Please try again later.")


In [26]:
def update_purpose_selection_keyboard(message, user_id, language):
    try:
        options = messages[language]['options']['purpose_visit']
        selected_options = user_data[user_id]['purpose_visit']
        
        inline_kb = types.InlineKeyboardMarkup(row_width=2)
        buttons = []
        for idx, option in enumerate(options):
            if option in selected_options:
                button_text = f"✅ {option}"
            else:
                button_text = option
            callback_data = f"purpose_{idx}"
            buttons.append(types.InlineKeyboardButton(text=button_text, callback_data=callback_data))
        
        done_button = types.InlineKeyboardButton(text=messages[language]['done_button'], callback_data="purpose_done")
        inline_kb.add(*buttons)
        inline_kb.add(done_button)
        
        bot.edit_message_reply_markup(chat_id=message.chat.id, message_id=message.message_id, reply_markup=inline_kb)
    except Exception as e:
        logging.exception(f"Error in update_purpose_selection_keyboard: {e}")
        bot.send_message(message.chat.id, messages[language].get('error_occurred', "An error occurred. Please try again later."))




def update_changes_detail_selection_keyboard(message, user_id, language):
    try:
        options = messages[language]['options']['changes_detail']
        selected_options = user_data[user_id]['changes_detail']
        
        inline_kb = types.InlineKeyboardMarkup(row_width=2)
        buttons = []
        for idx, option in enumerate(options):
            if option in selected_options:
                button_text = f"✅ {option}"
            else:
                button_text = option
            callback_data = f"changes_detail_{idx}"
            buttons.append(types.InlineKeyboardButton(text=button_text, callback_data=callback_data))
        
        done_button = types.InlineKeyboardButton(text=messages[language]['done_button'], callback_data="changes_detail_done")
        inline_kb.add(*buttons)
        inline_kb.add(done_button)
        
        bot.edit_message_reply_markup(chat_id=message.chat.id, message_id=message.message_id, reply_markup=inline_kb)
    except Exception as e:
        logging.exception(f"Error in update_changes_detail_selection_keyboard: {e}")
        bot.send_message(message.chat.id, messages[language].get('error_occurred', "An error occurred. Please try again later."))



In [27]:
def ask_age(chat_id, user_id, language):
    try:
        # Check if the user has already provided age
        if user_id in user_profiles and 'age' in user_profiles[user_id]:
            # Use previously stored age
            user_data[user_id]['age'] = user_profiles[user_id]['age']
            # Proceed to gender question
            ask_gender(chat_id, user_id, language)
            return

        options = messages[language]['options']['age']
        inline_kb = create_inline_keyboard(options, 'age')

        bot.send_message(
            chat_id,
            messages[language]['age_question'],
            reply_markup=inline_kb
        )
    except Exception as e:
        logging.exception(f"Error in ask_age: {e}")
        bot.send_message(chat_id, messages[language].get('error_occurred', "An error occurred. Please try again later."))


In [28]:
@bot.callback_query_handler(func=lambda call: call.data.startswith('age_'))
def handle_age_selection(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id
        language = user_data[user_id]['language']
        idx = int(call.data.split('_')[1])
        options = messages[language]['options']['age']

        if 0 <= idx < len(options):
            selected_age = options[idx]
            user_data[user_id]['age'] = selected_age
            user_profiles.setdefault(user_id, {})['age'] = selected_age

            # Acknowledge the selection
            bot.answer_callback_query(call.id, f"{messages[language]['selected']} {selected_age}")

            # Remove the inline keyboard
            bot.edit_message_reply_markup(chat_id=chat_id, message_id=call.message.message_id, reply_markup=None)

            # Echo the user's selection in italics
            bot.send_message(chat_id, f"<i>{escape_html(selected_age)}</i>", parse_mode='HTML')

            # Proceed to gender question
            ask_gender(chat_id, user_id, language)
        else:
            bot.answer_callback_query(call.id, messages[language].get('invalid_selection', "Invalid selection."))
    except Exception as e:
        logging.exception(f"Error in handle_age_selection: {e}")
        bot.send_message(chat_id, messages[language].get('error_occurred', "An error occurred. Please try again later."))


In [29]:
def ask_occupation(chat_id, user_id, language):
    try:
        # Check if the user has already provided occupation
        if user_id in user_profiles and 'occupation' in user_profiles[user_id]:
            # Use previously stored occupation
            user_data[user_id]['occupation'] = user_profiles[user_id]['occupation']
            # Proceed to income question
            ask_income(chat_id, user_id, language)
            return

        options = messages[language]['options']['occupation']
        inline_kb = create_inline_keyboard(options, 'occupation')

        bot.send_message(
            chat_id,
            messages[language]['occupation_question'],
            reply_markup=inline_kb
        )
    except Exception as e:
        logging.exception(f"Error in ask_occupation: {e}")
        bot.send_message(chat_id, messages[language].get('error_occurred', "An error occurred. Please try again later."))


In [30]:
@bot.callback_query_handler(func=lambda call: call.data.startswith('occupation_'))
def handle_occupation_selection(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id
        language = user_data[user_id]['language']
        idx = int(call.data.split('_')[1])
        options = messages[language]['options']['occupation']

        if 0 <= idx < len(options):
            selected_occupation = options[idx]
            user_data[user_id]['occupation'] = selected_occupation
            user_profiles.setdefault(user_id, {})['occupation'] = selected_occupation

            # Acknowledge the selection
            bot.answer_callback_query(call.id, f"{messages[language]['selected']} {selected_occupation}")

            # Remove the inline keyboard
            bot.edit_message_reply_markup(chat_id=chat_id, message_id=call.message.message_id, reply_markup=None)

            # Echo the user's selection in italics
            bot.send_message(chat_id, f"<i>{escape_html(selected_occupation)}</i>", parse_mode='HTML')

            # Proceed to income question
            ask_income(chat_id, user_id, language)
        else:
            bot.answer_callback_query(call.id, messages[language].get('invalid_selection', "Invalid selection."))
    except Exception as e:
        logging.exception(f"Error in handle_occupation_selection: {e}")
        bot.send_message(chat_id, messages[language].get('error_occurred', "An error occurred. Please try again later."))


In [31]:
def ask_gender(chat_id, user_id, language):
    try:
        # Check if the user has already provided gender
        if user_id in user_profiles and 'gender' in user_profiles[user_id]:
            # Use previously stored gender
            user_data[user_id]['gender'] = user_profiles[user_id]['gender']
            # Proceed to occupation question
            ask_occupation(chat_id, user_id, language)
            return

        options = messages[language]['options']['gender']
        inline_kb = create_inline_keyboard(options, 'gender')

        bot.send_message(
            chat_id,
            messages[language]['gender_question'],
            reply_markup=inline_kb
        )
    except Exception as e:
        logging.exception(f"Error in ask_gender: {e}")
        bot.send_message(
            chat_id,
            messages[language].get('error_occurred', "An error occurred. Please try again later.")
        )


In [32]:
@bot.callback_query_handler(func=lambda call: call.data.startswith('gender_'))
def handle_gender_selection(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id
        language = user_data[user_id]['language']
        data = call.data.split('_')[1]  # Should extract the index as string

        if data.isdigit():
            idx = int(data)
            options = messages[language]['options']['gender']

            if 0 <= idx < len(options):
                selected_gender = options[idx]
                user_data[user_id]['gender'] = selected_gender
                user_profiles.setdefault(user_id, {})['gender'] = selected_gender

                # Acknowledge the selection
                bot.answer_callback_query(call.id, f"{messages[language]['selected']} {selected_gender}")

                # Remove the inline keyboard
                bot.edit_message_reply_markup(
                    chat_id=chat_id,
                    message_id=call.message.message_id,
                    reply_markup=None
                )

                # Echo the user's selection in italics
                bot.send_message(chat_id, f"<i>{escape_html(selected_gender)}</i>", parse_mode='HTML')

                # Proceed to occupation question
                ask_occupation(chat_id, user_id, language)
            else:
                logging.warning(f"Invalid gender index {idx} for user {user_id}")
                bot.answer_callback_query(call.id, messages[language].get('invalid_selection', "Invalid selection."))
        else:
            logging.warning(f"Non-digit data in gender selection for user {user_id}: data={data}")
            bot.answer_callback_query(call.id, messages[language].get('invalid_selection', "Invalid selection."))
    except Exception as e:
        logging.exception(f"Error in handle_gender_selection: {e}")
        bot.send_message(
            chat_id,
            messages[language].get('error_occurred', "An error occurred. Please try again later.")
        )


In [33]:
def ask_income(chat_id, user_id, language):
    try:
        # Check if the user has already provided income
        if user_id in user_profiles and 'income' in user_profiles[user_id]:
            # Use previously stored income
            user_data[user_id]['income'] = user_profiles[user_id]['income']
            # Proceed to optional description
            ask_description(chat_id, user_id, language)
            return

        options = messages[language]['options']['income']
        inline_kb = create_inline_keyboard(options, 'income')

        bot.send_message(
            chat_id,
            messages[language]['income_question'],
            reply_markup=inline_kb
        )
    except Exception as e:
        logging.exception(f"Error in ask_income: {e}")
        bot.send_message(chat_id, messages[language].get('error_occurred', "An error occurred. Please try again later."))


In [34]:
@bot.callback_query_handler(func=lambda call: call.data.startswith('income_'))
def handle_income_selection(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id
        language = user_data[user_id]['language']
        data = call.data.split('_')[1]
        options = messages[language]['options']['income']

        if data.isdigit():
            idx = int(data)
            if 0 <= idx < len(options):
                selected_income = options[idx]
                user_data[user_id]['income'] = selected_income
                user_profiles.setdefault(user_id, {})['income'] = selected_income

                # Acknowledge the selection
                bot.answer_callback_query(call.id, f"{messages[language]['selected']} {selected_income}")

                # Remove the inline keyboard
                bot.edit_message_reply_markup(chat_id=chat_id, message_id=call.message.message_id, reply_markup=None)

                # Echo the user's selection in italics
                bot.send_message(chat_id, f"<i>{escape_html(selected_income)}</i>", parse_mode='HTML')

                # Proceed to optional description
                ask_description(chat_id, user_id, language)
            else:
                bot.answer_callback_query(call.id, messages[language].get('invalid_selection', "Invalid selection."))
        else:
            bot.answer_callback_query(call.id, messages[language].get('invalid_selection', "Invalid selection."))
    except NameError as e:
        logging.exception(f"NameError in handle_income_selection: {e}")
        bot.send_message(
            chat_id,
            "An internal error occurred. Please restart the survey.",
            reply_markup=types.ReplyKeyboardRemove()
        )
    except Exception as e:
        logging.exception(f"Error in handle_income_selection: {e}")
        bot.send_message(
            chat_id,
            messages[language].get('error_occurred', "An error occurred. Please try again later.")
        )


In [35]:
def ask_description(chat_id, user_id, language):
    try:
        # Use InlineKeyboard with a 'Skip' button
        inline_kb = types.InlineKeyboardMarkup()
        skip_button = types.InlineKeyboardButton(text=messages[language]['skip_button'], callback_data='description_skip')
        inline_kb.add(skip_button)
        
        bot.send_message(
            chat_id,
            messages[language]['add_description'],
            reply_markup=inline_kb
        )
        # The next step is to wait for text or voice message, or handle 'description_skip' callback
    except Exception as e:
        logging.exception(f"Error in ask_description: {e}")
        bot.send_message(chat_id, "An error occurred. Please try again later.")


In [36]:
@bot.callback_query_handler(func=lambda call: call.data == 'description_skip')
def handle_description_skip(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id
        language = user_data[user_id]['language']

        # Remove the inline keyboard
        bot.edit_message_reply_markup(chat_id=chat_id, message_id=call.message.message_id, reply_markup=None)

        # Notify user that they skipped the description
        bot.send_message(chat_id, messages[language]['description_skipped'])

        # Proceed to ask about lottery participation
        ask_lottery_participation(chat_id, user_id, language)
    except Exception as e:
        logging.exception(f"Error in handle_description_skip: {e}")
        bot.send_message(chat_id, "An error occurred. Please try again later.")


In [37]:
@bot.message_handler(content_types=['text', 'voice'])
def handle_description(message):
    try:
        chat_id = message.chat.id
        user_id = message.from_user.id

        language = user_data[user_id]['language']

        description = ''
        voice_submitted = ''  # Initialize as empty string

        # Remove the inline keyboard
        bot.edit_message_reply_markup(chat_id=chat_id, message_id=message.message_id - 1, reply_markup=None)

        # Check the type of message content
        if message.content_type == 'text':
            description = message.text.strip()
            # Echo the user's description in italics
            bot.send_message(chat_id, f"<i>{escape_html(description)}</i>", parse_mode='HTML')
        elif message.content_type == 'voice':
            # Handle voice message (existing code)
            # [Include your existing code for handling voice messages here]
            pass
        else:
            # If the content type is neither text nor voice, prompt the user again
            bot.send_message(
                chat_id,
                messages[language]['invalid_rating']
            )
            # Re-register the handler to wait for the correct input
            return

        # Store the description and voice_submitted in user_data
        user_data[user_id]['description'] = description
        user_data[user_id]['voice_submitted'] = voice_submitted

        # Proceed to ask about lottery participation
        ask_lottery_participation(chat_id, user_id, language)

    except Exception as e:
        logging.exception(f"Error in handle_description: {e}")
        bot.reply_to(message, "An error occurred. Please try again later.")


In [38]:
def ask_lottery_participation(chat_id, user_id, language):
    try:
        # Check if the user has already provided email
        if user_id in user_profiles and 'email' in user_profiles[user_id]:
            user_data[user_id]['email'] = user_profiles[user_id]['email']
            # Proceed to ask whether to continue or stop
            ask_continue_or_stop(chat_id, user_id, language)
            return

        options = messages[language]['lottery_options']
        inline_kb = create_inline_keyboard(options, 'lottery')

        bot.send_message(
            chat_id,
            messages[language]['lottery_question'],
            reply_markup=inline_kb
        )
    except Exception as e:
        logging.exception(f"Error in ask_lottery_participation: {e}")
        bot.send_message(
            chat_id, 
            messages[language].get('error_occurred', "An error occurred. Please try again later.")
        )


In [39]:
@bot.callback_query_handler(func=lambda call: call.data.startswith('lottery_'))
def handle_lottery_selection(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id
        language = user_data[user_id]['language']
        data = call.data.split('_')[1]

        options = messages[language]['lottery_options']

        if data == '0':  # Yes
            user_data[user_id]['lottery'] = options[0]
            user_profiles.setdefault(user_id, {})['lottery'] = options[0]

            # Acknowledge the selection
            bot.answer_callback_query(call.id, f"Selected: {options[0]}")

            # Echo the user's selection in italics
            if language == 'en':
                echo_text = f"You selected: <i>{options[0]}</i>"
            elif language == 'uk':
                echo_text = f"Ви обрали: <i>{options[0]}</i>"
            else:
                echo_text = f"You selected: <i>{options[0]}</i>"  # Default to English

            bot.send_message(chat_id, echo_text, parse_mode='HTML')

            # Remove the inline keyboard
            bot.edit_message_reply_markup(chat_id=chat_id, message_id=call.message.message_id, reply_markup=None)

            # Proceed to ask for email
            msg = bot.send_message(
                chat_id,
                messages[language]['email_request']
            )
            bot.register_next_step_handler(msg, handle_email)
        elif data == '1':  # No
            user_data[user_id]['lottery'] = options[1]
            user_profiles.setdefault(user_id, {})['lottery'] = options[1]

            # Acknowledge the selection
            bot.answer_callback_query(call.id, f"Selected: {options[1]}")

            # Echo the user's selection in italics
            if language == 'en':
                echo_text = f"You selected: <i>{options[1]}</i>"
            elif language == 'uk':
                echo_text = f"Ви обрали: <i>{options[1]}</i>"
            else:
                echo_text = f"You selected: <i>{options[1]}</i>"  # Default to English

            bot.send_message(chat_id, echo_text, parse_mode='HTML')

            # Remove the inline keyboard
            bot.edit_message_reply_markup(chat_id=chat_id, message_id=call.message.message_id, reply_markup=None)

            # Proceed to ask whether to continue or stop
            ask_continue_or_stop(chat_id, user_id, language)
        else:
            bot.answer_callback_query(call.id, messages[language].get('invalid_selection', "Invalid selection."))
    except Exception as e:
        logging.exception(f"Error in handle_lottery_selection: {e}")
        bot.send_message(chat_id, messages[language].get('error_occurred', "An error occurred. Please try again later."))


In [40]:
def handle_email(message):
    try:
        chat_id = message.chat.id
        user_id = message.from_user.id
        email = message.text.strip()
        language = user_data[user_id]['language']

        # Simple email validation
        email_regex = r'^\S+@\S+\.\S+$'
        if re.match(email_regex, email):
            user_data[user_id]['email'] = email
            # Store email in user_profiles
            user_profiles.setdefault(user_id, {})['email'] = email

            # Acknowledge and echo the email in italics
            echo_text = messages[language]['email_recorded'].format(email=f"<i>{escape_html(email)}</i>")
            bot.send_message(chat_id, echo_text, parse_mode='HTML')

            # Proceed to ask whether to continue or stop
            ask_continue_or_stop(chat_id, user_id, language)

        else:
            prompt_text = messages[language]['invalid_email']
            bot.send_message(chat_id, prompt_text)
            bot.register_next_step_handler_by_chat_id(chat_id, handle_email)
    except Exception as e:
        logging.exception(f"Error in handle_email: {e}")
        bot.send_message(
            chat_id,
            messages[language].get('error_occurred', "An error occurred. Please try again later.")
        )


In [41]:
def ask_continue_or_stop(chat_id, user_id, language):
    try:
        options = messages[language]['continue_options']
        inline_kb = types.InlineKeyboardMarkup(row_width=2)
        buttons = [
            types.InlineKeyboardButton(text=option, callback_data=f"continue_{idx}")
            for idx, option in enumerate(options)
        ]
        inline_kb.add(*buttons)
        bot.send_message(
            chat_id,
            messages[language]['continue_question'],
            reply_markup=inline_kb
        )
    except Exception as e:
        logging.exception(f"Error in ask_continue_or_stop: {e}")
        bot.send_message(chat_id, "An error occurred. Please try again later.")


In [42]:
@bot.callback_query_handler(func=lambda call: call.data.startswith('continue_'))
def handle_continue_or_stop_selection(call):
    try:
        chat_id = call.message.chat.id
        user_id = call.from_user.id
        language = user_data[user_id]['language']
        data = call.data.split('_')[1]
        options = messages[language]['continue_options']

        if data == '0':  # Continue
            # Acknowledge the selection
            bot.answer_callback_query(call.id, f"{messages[language]['selected']} {options[0]}")

            # Remove the inline keyboard
            bot.edit_message_reply_markup(chat_id=chat_id, message_id=call.message.message_id, reply_markup=None)

            # Save current data
            save_data_and_restart(chat_id, user_id, language, restart_survey=False)

            # Proceed to send location request (start a new experience)
            bot.send_message(chat_id, messages[language]['send_location'])

        elif data == '1':  # Stop
            # Acknowledge the selection
            bot.answer_callback_query(call.id, f"{messages[language]['selected']} {options[1]}")

            # Remove the inline keyboard
            bot.edit_message_reply_markup(chat_id=chat_id, message_id=call.message.message_id, reply_markup=None)

            # Send the thank you message without repeating the question
            if user_data[user_id].get('lottery') == messages[language]['lottery_options'][0]:
                bot.send_message(chat_id, messages[language]['thank_you_lottery'], reply_markup=types.ReplyKeyboardRemove())
            else:
                bot.send_message(chat_id, messages[language]['thank_you_no_lottery'], reply_markup=types.ReplyKeyboardRemove())

            # Proceed to save data
            save_data_and_restart(chat_id, user_id, language, restart_survey=False)

            # Send the 'consent_denied' message with the restart button
            restart_url = f"https://t.me/{bot_username}?start=restart"
            inline_kb = types.InlineKeyboardMarkup()
            restart_button = types.InlineKeyboardButton(
                text=messages[language]['restart_button'],
                url=restart_url
            )
            inline_kb.add(restart_button)
            bot.send_message(
                chat_id,
                messages[language]['consent_denied'],
                reply_markup=inline_kb
            )

        else:
            bot.answer_callback_query(call.id, messages[language].get('invalid_selection', "Invalid selection."))

    except Exception as e:
        logging.exception(f"Error in handle_continue_or_stop_selection: {e}")
        bot.send_message(chat_id, "An error occurred. Please try again later.")


In [43]:
def save_data_and_restart(chat_id, user_id, language, restart_survey=False):
    try:
        # Initialize Fernet for encryption
        fernet = Fernet(ENCRYPTION_KEY_GLOBAL)

        # Generate an anonymized user ID using SHA-1
        hash_object = hashlib.sha1(str(user_id).encode())
        anonymized_user_id = hash_object.hexdigest()

        # Prepare the data dictionary using .get() to handle missing fields
        data = {
            'user_id': anonymized_user_id,
            'latitude': str(user_data[user_id].get('location', {}).get('latitude', '')),
            'longitude': str(user_data[user_id].get('location', {}).get('longitude', '')),
            'enjoyment': user_data[user_id].get('enjoyment', ''),
            'purpose_visit': ';'.join(user_data[user_id].get('purpose_visit', [])),
            'regularity': user_data[user_id].get('regularity', ''),
            'frequency_change': user_data[user_id].get('frequency_change', ''),
            'noticed_changes': user_data[user_id].get('noticed_changes', ''),
            'changes_detail': ';'.join(user_data[user_id].get('changes_detail', [])),
            'other_changes_detail': user_data[user_id].get('other_changes_detail', ''),
            'travel_time': user_data[user_id].get('travel_time', ''),
            'description': user_data[user_id].get('description', ''),
            'voice_submitted': user_data[user_id].get('voice_submitted', ''),
            'age': user_data[user_id].get('age', ''),
            'gender': user_data[user_id].get('gender', ''),
            'occupation': user_data[user_id].get('occupation', ''),
            'income': user_data[user_id].get('income', ''),
            'language': language,
            'email': user_data[user_id].get('email', user_profiles.get(user_id, {}).get('email', '')),
            'timestamp': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        # List of fields to encrypt
        fields_to_encrypt = [
            'user_id', 'latitude', 'longitude', 'enjoyment', 'purpose_visit',
            'regularity', 'frequency_change', 'noticed_changes', 'changes_detail', 'other_changes_detail',
            'travel_time',
            'description', 'voice_submitted', 'age', 'gender', 'occupation', 'income',
            'language', 'email'
        ]

        # Encrypt the specified fields
        for field in fields_to_encrypt:
            if data[field]:
                data[field] = fernet.encrypt(data[field].encode()).decode()
            else:
                data[field] = ''  # Ensure the field is a string even if empty

        # Insert the data into the database
        with db_lock:
            conn = sqlite3.connect(db_file, check_same_thread=False)
            cursor = conn.cursor()

            cursor.execute('''
                INSERT INTO responses (
                    user_id, latitude, longitude, enjoyment, purpose_visit,
                    regularity, frequency_change, noticed_changes, changes_detail, other_changes_detail,
                    travel_time,
                    description, voice_submitted, age, gender, occupation, income,
                    language, email, timestamp
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                data['user_id'],
                data['latitude'],
                data['longitude'],
                data['enjoyment'],
                data['purpose_visit'],
                data['regularity'],
                data['frequency_change'],
                data['noticed_changes'],
                data['changes_detail'],
                data['other_changes_detail'],
                data['travel_time'],
                data['description'],
                data['voice_submitted'],
                data['age'],
                data['gender'],
                data['occupation'],
                data['income'],
                data['language'],
                data['email'],
                data['timestamp']
            ))
            conn.commit()
            conn.close()

        # Clear only the current experience data
        experience_keys = [
            'location', 'enjoyment', 'purpose_visit', 'regularity', 'frequency_change', 'noticed_changes',
            'changes_detail', 'other_changes_detail', 'travel_time', 'description', 'voice_submitted'
        ]
        for key in experience_keys:
            user_data[user_id].pop(key, None)

        if restart_survey:
            # Restart the survey for the user
            send_welcome(chat_id=chat_id, user_id=user_id)
    except Exception as e:
        logging.exception(f"Error in save_data_and_restart: {e}")
        bot.send_message(
            chat_id,
            messages[language].get('error_occurred', "An error occurred while saving your data. Please try again later.")
        )


In [ ]:
if __name__ == '__main__':
    # Initialize the database
    initialize_database()

    # Start the bot
    while True:
        try:
            bot.polling(none_stop=True)
        except Exception as e:
            logging.exception(f"Bot polling failed: {e}")
            time.sleep(15)
